In [1]:
import numpy as np
from itertools import combinations
from scipy.spatial.distance import cdist
from scipy.optimize import linprog
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix

In [2]:
# Define helperfuntions
# Step 4: Check for disconnected components (subtours)
def check_subtours(moat_widths, subsets, n):
    # Construct an adjacency matrix based on the moat widths
    adjacency_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            moat_sum = 0
            for k, S in enumerate(subsets):
                if (i in S and j not in S) or (i not in S and j in S):
                    moat_sum += moat_widths[k]
            adjacency_matrix[i, j] = moat_sum
            adjacency_matrix[j, i] = moat_sum
    
    # Use sparse graph representation to check connected components
    graph = csr_matrix(adjacency_matrix > 0)  # Convert to binary graph (edges exist or not)
    num_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    
    if num_components > 1:
        # Return the subtours (disconnected components)
        subtours = [np.where(labels == i)[0] for i in range(num_components)]
        return subtours
    else:
        return None

# Step 5: Implement moat nesting
def nest_moats(moat_widths, subsets):
    nested_widths = moat_widths.copy()
    
    # Sort the moats by size (smaller subsets first)
    sorted_subsets = sorted(enumerate(subsets), key=lambda x: len(x[1]))
    
    for i, (k, subset) in enumerate(sorted_subsets):
        for j, (k_other, larger_subset) in enumerate(sorted_subsets):
            if subset.issubset(larger_subset) and i != j:
                # Instead of subtracting, we ensure smaller moats are nested inside larger moats
                nested_widths[k_other] = max(nested_widths[k_other], moat_widths[k] + nested_widths[k_other])
    
    return nested_widths

# Step 6: Recalculate the cost allocation
def calculate_cost_allocation(nested_widths, subsets, n):
    allocation = [0] * n
    total_moat_width = sum(nested_widths)
    
    # Allocate costs based on participation in moats
    for i in range(n):
        for k, S in enumerate(subsets):
            if i in S:
                allocation[i] += nested_widths[k] / len(S)
    
    # Normalize allocation to ensure the total cost is the MPV
    total_allocation = sum(allocation)
    if total_allocation > 0:
        allocation = [a * total_moat_width / total_allocation for a in allocation]
    
    return allocation

In [3]:
# Define the distance matrix (this will be dynamically generated in practice)
distance_matrix = np.array([
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]])

# Define the number of locations
n = len(distance_matrix)

# Step 1: Generate all possible subsets (S, S') of the locations N (except 0)
subsets = []
for size in range(1, n):
    subsets += [set(combo) for combo in combinations(range(1, n), size)]

num_subsets = len(subsets)

# Objective: maximize the sum of moat widths
c = [-1] * num_subsets  # Negative because linprog minimizes, we want to maximize

# Bounds: moat widths must be >= 0
bounds = [(0, None)] * num_subsets

# Step 2: Prepare the initial constraints for moat widths
A = []
b = []

for i in range(n):
    for j in range(i + 1, n):
        constraint = [0] * num_subsets
        for k, S in enumerate(subsets):
            if (i in S and j not in S) or (i not in S and j in S):
                constraint[k] = 1
        A.append(constraint)
        b.append(distance_matrix[i][j])

# Convert A and b into numpy arrays for compatibility with scipy's linprog
A = np.array(A)
b = np.array(b)

# Step 3: Solve the initial linear program
res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method="highs")

# Step 7: Solve the problem and allocate costs

# Initial solution
moat_widths = res.x

# Check for subtours
subtours = check_subtours(moat_widths, subsets, n)
while subtours:
    # Add subtour constraints (if any) and re-solve the linear program
    for subtour in subtours:
        new_constraint = [0] * num_subsets
        for k, S in enumerate(subsets):
            if any(i in S for i in subtour) and any(i not in S for i in subtour):
                new_constraint[k] = 1
        A = np.vstack([A, new_constraint])
        b = np.append(b, 2)  # Subtour constraint
    
    res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method="highs")
    moat_widths = res.x
    subtours = check_subtours(moat_widths)

# Perform moat nesting
nested_widths = nest_moats(moat_widths, subsets)

# Calculate cost allocation based on nested moats
allocation = calculate_cost_allocation(nested_widths, subsets, n)

# Output results
print("Moat widths:", nested_widths)
print("Maximum Packing Value (MPV):", sum(nested_widths))
print("Cost allocation per location:", allocation)

Moat widths: [ 7.5 12.5 15.  20.  25.  30.  40. ]
Maximum Packing Value (MPV): 150.0
Cost allocation per location: [0.0, 43.333333333333336, 50.833333333333336, 55.833333333333336]
